In [9]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

In [10]:
# Step 1: Create an imbalanced binary classification dataset
X, y = make_classification(n_samples=1000, n_features=10, n_informative=2, n_redundant=8, 
                           weights=[0.9, 0.1], flip_y=0, random_state=42)

np.unique(y, return_counts=True)

(array([0, 1]), array([900, 100], dtype=int64))

In [11]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

In [12]:
params={
    "solver":"lbfgs",
    "max_iter":1000,
    "multi_class":"auto",
    "random_state":8888,
}
lr=LogisticRegression(**params)
lr.fit(X_train,y_train)
y_pred=lr.predict(X_test)
report=classification_report(y_test,y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.95      0.97      0.96       270
           1       0.62      0.50      0.56        30

    accuracy                           0.92       300
   macro avg       0.79      0.73      0.76       300
weighted avg       0.91      0.92      0.92       300



In [14]:
report_dict=classification_report(y_test,y_pred,output_dict=True)
report_dict


{'0': {'precision': 0.9456521739130435,
  'recall': 0.9666666666666667,
  'f1-score': 0.9560439560439561,
  'support': 270.0},
 '1': {'precision': 0.625,
  'recall': 0.5,
  'f1-score': 0.5555555555555556,
  'support': 30.0},
 'accuracy': 0.92,
 'macro avg': {'precision': 0.7853260869565217,
  'recall': 0.7333333333333334,
  'f1-score': 0.7557997557997558,
  'support': 300.0},
 'weighted avg': {'precision': 0.9135869565217392,
  'recall': 0.92,
  'f1-score': 0.9159951159951161,
  'support': 300.0}}

In [13]:
import mlflow

In [17]:
mlflow.set_experiment("First experiment")
mlflow.set_tracking_uri("http://127.0.0.1:5000/")

with mlflow.start_run():
    mlflow.log_params(params)
    mlflow.log_metrics({
        "accuracy":report_dict['accuracy'],
        "recall_class_0":report_dict["0"]["recall"],
        "recall_class_1":report_dict["1"]["recall"],
        "f1_score_macro":report_dict['macro avg']['f1-score']
    })
    mlflow.sklearn.log_model(lr,"Logistic Regression")

2024/11/27 10:30:38 INFO mlflow.tracking.fluent: Experiment with name 'First experiment' does not exist. Creating a new experiment.
2024/11/27 10:30:45 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run dapper-cod-248 at: http://127.0.0.1:5000/#/experiments/702216214394275621/runs/1330d2fec18c44a5bba90dc43e2f85a6
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/702216214394275621


In [18]:
log_reg = LogisticRegression(C=1, solver='liblinear')
log_reg.fit(X_train, y_train)
y_pred_log_reg = log_reg.predict(X_test)
print(classification_report(y_test, y_pred_log_reg))

              precision    recall  f1-score   support

           0       0.95      0.96      0.95       270
           1       0.60      0.50      0.55        30

    accuracy                           0.92       300
   macro avg       0.77      0.73      0.75       300
weighted avg       0.91      0.92      0.91       300



In [19]:
rf_clf = RandomForestClassifier(n_estimators=30, max_depth=3)
rf_clf.fit(X_train, y_train)
y_pred_rf = rf_clf.predict(X_test)
print(classification_report(y_test, y_pred_rf))

              precision    recall  f1-score   support

           0       0.96      1.00      0.98       270
           1       0.95      0.67      0.78        30

    accuracy                           0.96       300
   macro avg       0.96      0.83      0.88       300
weighted avg       0.96      0.96      0.96       300



In [20]:
xgb_clf = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_clf.fit(X_train, y_train)
y_pred_xgb = xgb_clf.predict(X_test)
print(classification_report(y_test, y_pred_xgb))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99       270
           1       0.96      0.80      0.87        30

    accuracy                           0.98       300
   macro avg       0.97      0.90      0.93       300
weighted avg       0.98      0.98      0.98       300



In [21]:
from imblearn.combine import SMOTETomek

smt = SMOTETomek(random_state=42)
X_train_res, y_train_res = smt.fit_resample(X_train, y_train)

np.unique(y_train_res, return_counts=True)

(array([0, 1]), array([619, 619], dtype=int64))

In [22]:
xgb_clf = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_clf.fit(X_train_res, y_train_res)
y_pred_xgb = xgb_clf.predict(X_test)
print(classification_report(y_test, y_pred_xgb))

              precision    recall  f1-score   support

           0       0.98      0.98      0.98       270
           1       0.81      0.83      0.82        30

    accuracy                           0.96       300
   macro avg       0.89      0.91      0.90       300
weighted avg       0.96      0.96      0.96       300



In [30]:
models = [
    (
        "Logistic Regression", 
        {"C": 1, "solver": 'liblinear'},
        LogisticRegression(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "Random Forest", 
        {"n_estimators": 30, "max_depth": 3},
        RandomForestClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier With SMOTE",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(), 
        (X_train_res, y_train_res),
        (X_test, y_test)
    )
]

In [31]:
reports = []

for model_name, params, model, train_set, test_set in models:
    X_train = train_set[0]
    y_train = train_set[1]
    X_test = test_set[0]
    y_test = test_set[1]
    
    model.set_params(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    reports.append(report)

In [32]:
reports

[{'0': {'precision': 0.9834710743801653,
   'recall': 0.8814814814814815,
   'f1-score': 0.9296875,
   'support': 270.0},
  '1': {'precision': 0.4482758620689655,
   'recall': 0.8666666666666667,
   'f1-score': 0.5909090909090909,
   'support': 30.0},
  'accuracy': 0.88,
  'macro avg': {'precision': 0.7158734682245654,
   'recall': 0.8740740740740741,
   'f1-score': 0.7602982954545454,
   'support': 300.0},
  'weighted avg': {'precision': 0.9299515531490453,
   'recall': 0.88,
   'f1-score': 0.8958096590909091,
   'support': 300.0}},
 {'0': {'precision': 0.988,
   'recall': 0.9148148148148149,
   'f1-score': 0.95,
   'support': 270.0},
  '1': {'precision': 0.54, 'recall': 0.9, 'f1-score': 0.675, 'support': 30.0},
  'accuracy': 0.9133333333333333,
  'macro avg': {'precision': 0.764,
   'recall': 0.9074074074074074,
   'f1-score': 0.8125,
   'support': 300.0},
  'weighted avg': {'precision': 0.9431999999999999,
   'recall': 0.9133333333333333,
   'f1-score': 0.9225,
   'support': 300.0}}

In [35]:
import mlflow

In [37]:
# Initialize MLflow
mlflow.set_experiment("AnomalyDetection")
mlflow.set_tracking_uri("http://localhost:5000")

for i, element in enumerate(models):
    model_name = element[0]
    params = element[1]
    model = element[2]
    report = reports[i]
    
    with mlflow.start_run(run_name=model_name):        
        mlflow.log_params(params)
        mlflow.log_metrics({
            'accuracy': report['accuracy'],
            'recall_class_1': report['1']['recall'],
            'recall_class_0': report['0']['recall'],
            'f1_score_macro': report['macro avg']['f1-score']
        })  
        
        if "XGB" in model_name:
            mlflow.xgboost.log_model(model, "model")
        else:
            mlflow.sklearn.log_model(model, "model")  

2024/11/27 11:19:26 INFO mlflow.tracking.fluent: Experiment with name 'AnomalyDetection' does not exist. Creating a new experiment.
2024/11/27 11:19:31 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Logistic Regression at: http://localhost:5000/#/experiments/708238129382033550/runs/8c4b0d3c9a3541c693a0e11a62c3fd80
🧪 View experiment at: http://localhost:5000/#/experiments/708238129382033550


2024/11/27 11:19:34 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Random Forest at: http://localhost:5000/#/experiments/708238129382033550/runs/6327873a5ae14c08a4765c4ba026001d
🧪 View experiment at: http://localhost:5000/#/experiments/708238129382033550


2024/11/27 11:19:36 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run XGBClassifier at: http://localhost:5000/#/experiments/708238129382033550/runs/8594ba69ff8d48858830052949c7663b
🧪 View experiment at: http://localhost:5000/#/experiments/708238129382033550


2024/11/27 11:19:38 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run XGBClassifier With SMOTE at: http://localhost:5000/#/experiments/708238129382033550/runs/1d9f4ef96b274d7faa13126689cb70f1
🧪 View experiment at: http://localhost:5000/#/experiments/708238129382033550


In [40]:
## Register the model
model_name = 'XGB-Smote'
run_id=input('Please type RunID')
model_uri = f'runs:/{run_id}/model'

with mlflow.start_run(run_id=run_id):
    mlflow.register_model(model_uri=model_uri, name=model_name)

Please type RunID 1d9f4ef96b274d7faa13126689cb70f1


Successfully registered model 'XGB-Smote'.
2024/11/27 11:40:15 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: XGB-Smote, version 1


🏃 View run XGBClassifier With SMOTE at: http://localhost:5000/#/experiments/708238129382033550/runs/1d9f4ef96b274d7faa13126689cb70f1
🧪 View experiment at: http://localhost:5000/#/experiments/708238129382033550


Created version '1' of model 'XGB-Smote'.


In [41]:
model_version = 1
model_uri = f"models:/{model_name}/{model_version}"

loaded_model = mlflow.xgboost.load_model(model_uri)
y_pred = loaded_model.predict(X_test)
y_pred[:4]

array([0, 0, 0, 0])

In [44]:
current_model_uri = f"models:/{model_name}@challenger"
production_model_name = "anomaly-detection-prod"

client = mlflow.MlflowClient()
client.copy_model_version(src_model_uri=current_model_uri, dst_name=production_model_name)

Successfully registered model 'anomaly-detection-prod'.
Copied version '1' of model 'XGB-Smote' to version '1' of model 'anomaly-detection-prod'.


<ModelVersion: aliases=[], creation_timestamp=1732688038889, current_stage='None', description='', last_updated_timestamp=1732688038889, name='anomaly-detection-prod', run_id='1d9f4ef96b274d7faa13126689cb70f1', run_link='', source='models:/XGB-Smote/1', status='READY', status_message='', tags={}, user_id='', version='1'>

In [45]:
model_version = 1
prod_model_uri = f"models:/{production_model_name}@champion"

loaded_model = mlflow.xgboost.load_model(prod_model_uri)
y_pred = loaded_model.predict(X_test)
y_pred[:4]

array([0, 0, 0, 0])